In [ ]:
%pip install huggingface_hub transformers peft bitsandbytes trl xformers datasets torch flash-attention

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
import yaml
import torch

In [ ]:
# Configurations
MODEL_ID = "microsoft/Phi-3.5-mini-instruct"
NEW_MODEL_NAME = "Phi-3.5-EGNIVIA"
DATASET_NAME = "/content/drive/My Drive/datasets/EGNIVIA-finetune-dataset"
SPLIT = "train"
MAX_SEQ_LENGTH = 2048
num_train_epochs = 1
license = "apache-2.0"
learning_rate = 1.41e-5
per_device_train_batch_size = 4
gradient_accumulation_steps = 1

if torch.cuda.is_bf16_supported():
    print("Using supported bfloat16")
    compute_dtype = torch.bfloat16
else:
    print("Using supported float16")
    compute_dtype = torch.float16

In [ ]:
# Load the model, tokenizer, and dataset
from google.colab import drive
drive.mount('/content/drive')

import os

# Set up the cache directory
cache_dir = "/content/drive/My Drive/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                                 cache_dir=cache_dir,
                                                 trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                              cache_dir=cache_dir,
                                              trust_remote_code=True)


In [ ]:
# Load and split dataset
dataset = load_dataset(DATASET_NAME, split="train")
train_size = int(len(dataset) * 0.9)

train_dataset = dataset.select(range(train_size))
eval_dataset = dataset.select(range(train_size, len(dataset)))

print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")

In [ ]:
# Preprocess the dataset
def formatting_prompts_func(examples):
    """
    Format examples using chat template, compatible with SFTTrainer
    """
    texts = []

    for i in range(len(examples['question'])):
        messages = [
            {"role": "system", "content": examples['context'][i]},
            {"role": "user", "content": examples['question'][i]},
            {"role": "assistant", "content": examples['answer'][i]}
        ]

        # Note: We need to create the tokenizer outside this function
        # since we don't want to load it repeatedly for each batch
        formatted = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )

        texts.append(formatted)

    return texts

In [ ]:
# Check a few examples
test_samples = {k: [dataset[i][k] for i in range(3)] for k in dataset[0].keys()}
formatted = formatting_prompts_func(test_samples)
for i, text in enumerate(formatted):
    print(f"\nExample {i}:")
    print("-" * 50)
    print(text)

In [ ]:
# Define the training arguments
args = TrainingArguments(
    eval_strategy="steps",
    per_device_train_batch_size=7,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    max_steps=-1,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_steps=10,
    output_dir=NEW_MODEL_NAME,
    optim="paged_adamw_32bit",
    lr_scheduler_type="linear",
)

# Create SFTConfig
sft_config = SFTConfig(
    output_dir=NEW_MODEL_NAME,
    dataset_text_field="text",
    max_seq_length=128,
)

In [ ]:
# Start the fine-tuning process
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    formatting_func=formatting_prompts_func,
    output_dir="/content/Phi-3.5-EGNIVIA/"
)
trainer.train()

In [ ]:
# Export the training data log
from google.colab import files

!zip -r wandb.zip /content/wandb/
files.download("wandb.zip")